<a href="https://colab.research.google.com/github/opencoca/vidsum/blob/master/GPT_J_6B_Inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GPT-J-6B Inference 

This notebook explores how to work with the [GPT-J-6B model](https://github.com/kingoflolz/mesh-transformer-jax/#GPT-J-6B). See the link for more details about the model, including evaluation metrics and credits.

Please note it takes ~10 minutes for this notebook to fully spin up. Sometimes you will runout of ram when attempting to setup the model. If this happens factory restart the runtime and try again. If it contines to happen ...

Start this notebok by clicking `Runtime > Run all` or clicking `⌘/Ctrl+F9`. Use `⌘` on Mac and `Ctrl` on Windows and Linux. 

Keep an eye on this as it spins up. Google likes to make sure you are not a bot and will stop things if you ignore the "I'm not a robot" popup.

## Prepare it to be used

### Install Dependencies

First we download the model and install some dependencies. This step takes at least 5 minutes (possibly longer depending on server load).

!!! **Make sure you are using a TPU runtime!** !!!

In [ ]:
#@title Mount Google Drive and clone project repo
#@markdown Not having to reinstall all libs speeds up relaunch immensely
!git clone -q https://github.com/kingoflolz/mesh-transformer-jax.git

import os, sys
from google.colab import drive
drive.mount('/content/drive')
nb_path = '/content/notebooks'
os.symlink('/content/drive/My Drive/Colab Notebooks', nb_path)
sys.path.insert(0,nb_path)

pk_path = '/content/site-packages'
os.makedirs('/content/drive/My Drive/Colab Notebooks/site-packages', exist_ok=True)
os.symlink('/content/drive/My Drive/Colab Notebooks/site-packages', pk_path)
sys.path.insert(0,pk_path)

os.symlink('/content/notebooks/step_383500_slim.tar.zstd', '/content/step_383500_slim.tar.zstd')

In [ ]:
#@title Install GPT-J-6B Network
#@markdown So long as you already have a copy of GPT-J in your Google Drive this is a fast process. If the is is the first time it can take almost half an hour if https://the-eye.eu is flooded with traffic.
!apt-get install -qq zstd

# the "slim" version contain only bf16 weights and no optimizer parameters, which minimizes bandwidth and memory
!echo "Getting https://the-eye.eu/public/AI/GPT-J-6B/step_383500_slim.tar.zstd this can take ~6-20 min."
!time wget  -nc -c https://the-eye.eu/public/AI/GPT-J-6B/step_383500_slim.tar.zstd

!time tar -I zstd -xf step_383500_slim.tar.zstd


In [ ]:
#@title Install Tensorflow
#@markdown restart once this has run

!pip install --target=$pk_path tensorflow==2.5.0

In [ ]:
#@title Install GPT-J-6B Inference OpenCo's requirements.txt
#@markdown restart once this has run

!wget -O mesh-transformer-jax/requirements.txt https://gist.githubusercontent.com/opencoca/ea36cfdf091c93df337f36951fa93a50/raw/83f7e5246a0c7ed873569e1edab93ef289252f1e/requirements.txt

!echo "Quietly installing GPT-J-6B Inference OpenCo's requirements.txt"
!pip -q install --target=$pk_path -r mesh-transformer-jax/requirements.txt

# jax 0.2.12 is required due to a regression with xmap in 0.2.13
!pip -q install --target=$pk_path mesh-transformer-jax/ jax==0.2.12
!pip -q install --target=$pk_path optax==0.0.9 transformers==4.12.3 dm-haiku==0.0.5 einops==0.3.2 jax==0.2.12 ray==1.8.0 


In [ ]:
#@title Re-Install Tensorflow
#@markdown restart once this has run

!pip install --target=$pk_path tensorflow==2.5.0

## Setup Model

Make sure to  restart your runtime for this will work reliably. 




In [ ]:
#@title Setup JAX for TPU use
import os
import requests 
from jax.config import config

colab_tpu_addr = os.environ['COLAB_TPU_ADDR'].split(':')[0]
url = f'http://{colab_tpu_addr}:8475/requestversion/tpu_driver0.1_dev20210607'
print(url)
requests.post(url)

# The following is required to use TPU Driver as JAX's backend.
config.FLAGS.jax_xla_backend = "tpu_driver"
config.FLAGS.jax_backend_target = "grpc://" + os.environ['COLAB_TPU_ADDR']

Sometimes the next step errors for some reason, just run it again ¯\\\_(ツ)\_/¯

In [ ]:
#@title Import libraries
#@markdown If this fails reinstall OpenCo's requirments and attempt Model Setup again.
import time

import jax
from jax.experimental import maps
import numpy as np
import optax
import transformers

from mesh_transformer.checkpoint import read_ckpt
from mesh_transformer.sampling import nucleaus_sample
from mesh_transformer.transformer_shard import CausalTransformer

In [ ]:
#@title Set Network Parameters 
params = {
  "layers": 28,
  "d_model": 4096,
  "n_heads": 16,
  "n_vocab": 50400,
  "norm": "layernorm",
  "pe": "rotary",
  "pe_rotary_dims": 64,

  "seq": 2048,
  "cores_per_replica": 8,
  "per_replica_batch": 1,
}

per_replica_batch = params["per_replica_batch"]
cores_per_replica = params["cores_per_replica"]
seq = params["seq"]


params["sampler"] = nucleaus_sample

# here we "remove" the optimizer parameters from the model (as we don't need them for inference)
params["optimizer"] = optax.scale(0)

mesh_shape = (jax.device_count() // cores_per_replica, cores_per_replica)
devices = np.array(jax.devices()).reshape(mesh_shape)

maps.thread_resources.env = maps.ResourceEnv(maps.Mesh(devices, ('dp', 'mp')))

tokenizer = transformers.GPT2TokenizerFast.from_pretrained('gpt2')

In [ ]:
#@title Install Tracery 
#@markdown Tracery is a gramar for spining articles, stories, and text in general. https://github.com/aparrish/pytracery
!pip install tracery

In [ ]:
#@title Tidy things up
from IPython.display import clear_output
clear_output()

In [ ]:
#@title Create the network
#@markdown ...loading neural weights from the downloaded files. 
#@markdown
#@markdown  > *This can take around many minutes.*
total_batch = per_replica_batch * jax.device_count() // cores_per_replica

network = CausalTransformer(params)

network.state = read_ckpt(network.state, "step_383500/", devices.shape[1])

network.state = network.move_xmap(network.state, np.zeros(cores_per_replica))

# Run Model

Finally, we are ready to infer with the model! The first sample takes around a minute due to compilation, but after that it should only take about 10 seconds per sample.

Feel free to mess with the different sampling parameters (top_p and temp), as well as the length of the generations (gen_len, causes a recompile when initially changed - recompiles will be cached).

You can also change other things like per_replica_batch in the previous cells to change how many generations are done in parallel. A larger batch has higher latency but higher throughput when measured in tokens generated/s. This is useful for doing things like best-of-n cherry picking.

*Tip for best results: Make sure your prompt does not have any trailing spaces, which tend to confuse the model due to the BPE tokenization used during training.*

In [ ]:
#@markdown We allow for text wrapping of whitespace via css injection.

# allow text wrapping in generated output: https://stackoverflow.com/a/61401455
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

In [ ]:
#@title Define our infer function. 
def infer(context, top_p=0.9, temp=1.0, gen_len=256):
    tokens = tokenizer.encode(context)

    provided_ctx = len(tokens)
    pad_amount = seq - provided_ctx

    padded_tokens = np.pad(tokens, ((pad_amount, 0),)).astype(np.uint32)
    batched_tokens = np.array([padded_tokens] * total_batch)
    length = np.ones(total_batch, dtype=np.uint32) * len(tokens)

    start = time.time()
    output = network.generate(batched_tokens, length, gen_len, {"top_p": np.ones(total_batch) * top_p, "temp": np.ones(total_batch) * temp})

    samples = []
    decoded_tokens = output[1][0]

    for o in decoded_tokens[:, :, 0]:
      samples.append(f"\033[1m{context}\033[0m{tokenizer.decode(o)}")

    #print(f"completion done in {time.time() - start:06}s")
    #this should instead be added to the object returned infered.samples and infered.completion_time
    return samples




In [ ]:
#@title Warmup our inference engine 
#@markdown The first time that infrences are run for a given length takes imensly longer than on repeated runs. For this reason we spin up the inference engine with a few sample runs. With this out of the way we can quickly experiment with our infrence engine. 

context = """
# A NEW WAY TO WORK IN THE OLD PORT OF MONTREAL
## SPACES CITÉ MULTIMÉDIA

The Old Port of Montreal is a"""

for i in [32,64,96,128,256,512,1024,2048]:
  print("- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -")
  print(f" ## Infering context with a gen_len of {i} this should take ~{i/24} sec. when run again ##")
  print("- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -")
  print(infer(context, gen_len=i)[0])

# Example prompts

We've compiled the model. At this point it should only take about 15 seconds per sample. The first time you run inference on a new dataset takes longer than usual because of compilation to build the graph and write data into disk buffers for fast access by CPUs/GPUs.

The following parameters can be adjusted: top_p, temp, gen_len. When changing the length of generations (gen_len), recompilation is required to maintain accuracy and consistency in results.

You can also change other things like the number of generations done in parallel and how many samples are generated. In addition, you can increase or decrease the batch size for latency/throughput tradeoffs as well. This is useful when performing best-of-n cherry picking tasks where throughput needs to be maximized with minimum batch time delay on each sample generation process.

To get the best results, make sure your prompt does not have any trailing spaces. This is because BPE tokenization can be confused by them.

## Simple Inference 

### Set Sampling Parameters


In [ ]:
#@markdown ### Leave this alone unless you know what you are doing! 
#@markdown ... or you are just having fun experimenting and don't mind having weird results.
#@markdown - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
#@markdown Top_p controls diversity/quality. The higher our top_p the more rich and nuanced oir inferences can be.
top_p = 1 #@param {type:"slider", min:0, max:1, step:0.1}
#@markdown - - - 
#@markdown Temp controls randomnes. Low temps make for mechanical inferences.
temp = 0.9 #@param {type:"slider", min:0, max:1, step:0.1}
#@markdown - - - 
#@markdown Gen_len adjects the length of text infered.
#@markdown Specifcily it adustes the nember of tokens (~4 per engish word).
#@markdown
#@markdown *Note: Changing gen_len will cause long wait times when first setting lengs that have not been set before.*
#@markdown  
gen_len = 64 #@param {type: "slider", min:64, max: 1024, step: 64}
#@markdown - - - 
#@markdown Generate controls the number of inferences that will be generated.
generate = 1 #@param {type: "slider", min:1, max: 12, step: 1}


### Text examples

In [ ]:
#@title Raw Context 
#@markdown This is an example of simple summery infrence.

#print("Enter/Paste your content. Ctrl-D or Ctrl-Z ( windows ) to save it.")
#context = []
#while True:
#    try:
#        line = input()
#    except EOFError:
#        break
#    context.append(line)

context = """"Some analysts expect bitcoin to eventually catch up to the rise in altcoins next week, which could push BTC’s price past $64,000 resistance.

“Historically, there has been a slightly delayed positive correlation with traditional markets and the crypto market, helping to build the case for a bullish November for digital assets,” Will Morris, a trader at the U.K.-based digital asset broker GlobalBlock, wrote in an email to CoinDesk.

Latest prices
Bitcoin (BTC): 60,956.33, -0.30%
Ether (ETH): 4,484.81, -0.01%
S&P 500: 4,697.53, +0.37%
Gold: 1,817.17, +1.44%
10-year Treasury yield closed at 1.45%
Traders will also be monitoring Coinbase’s (NASDAQ: COIN) third-quarter earnings report on Nov. 9. The crypto exchange delivered a negative earnings surprise in the second quarter because of declining trading volume. The cryptocurrency exchange’s shares are up about 40% over the past month compared to a 10% rise in bitcoin and a 25% rise in ether over the same period.


Altcoin roundup
Bitcoin cash briefly spikes on fraudulent press release: Bitcoin cash (BCH) rose a sharp 4.6% to $630.70 in under 15 minutes on Friday after a fraudulent press release went public, CoinDesk’s Jamie Crawley reported. The release stated that U.S. supermarket giant Kroger would start accepting the cryptocurrency as a form of payment during the year-end holiday season this year. However, the news – which was issued on PR Newswire and appeared on Kroger’s website – was quickly taken down after a spokesperson confirmed that it was fake news. The BCH price declined again and stood at $601.74 at press time.
Argo blockchain shares fall after workers share non-public information: Shares of London-based crypto miner Argo Blockchain (ARBK) fell as much as 5% on Friday after the company said in a filing that some employees had inadvertently disclosed potentially material, non-public information in a conversation, CoinDesk’s Aoyon Ashraf reported. The discussion contained information about the potential increase in the company’s hashrate, or computing power, and the expected cost to build its planned facility in Texas, according to the filing. Argo listed its American depositary shares on Nasdaq in September; they are up roughly 3% since the listing.
FTX, Lightspeed, Solana Ventures to invest $100 million in Web 3 gaming: FTX, Lightspeed and Solana Ventures are investing $100 million in Web 3 gaming development, CoinDesk’s Eli Tan reported. The funding will go towards the development of new titles as well as help with the integration of Solana’s blockchain into existing games, creating in-game economies centered around non-fungible tokens (NFT) and Solana wallet payments. The initiative has already made a first investment, with FTX and Lightspeed leading a $21 million funding round for gaming studio Faraway.

Summary: ETH is"""

inference = infer(top_p=top_p, temp=0.96, gen_len=128, context=context)[0]

print(inference.split("Summary:")[-1].strip())

In [ ]:
#@title Expand into a script 
#@markdown The will expand a lead into a script bassed on the opening of pulp fiction.
write_script_on_this = "Feline biology is amazing" #@param {type:"string"}

#@markdown - - - - - - - - - - - - - - - - - - - - - - - - -
#@markdown These options don't need to be adjusted.
top_p = 0.9 #@ param {type:"slider", min:0, max:1, step:0.1}
temp = 0.9 #@ param {type:"slider", min:0, max:1, step:0.1}
gen_len = 512 #@param {type: "slider", min:64, max: 2048, step: 64}
generate = 1 #@ param {type: "slider", min:1, max: 12, step: 1}
#@markdown - - - - - - - - - - - - - - - - - - - - - - - - -


context = """

INT. COMPUTER LAB – MORNING

A normal computer lab, an office or university, Toronto Canada.
It's about 9:00 in the morning. While the place isn't jammed,
there's a healthy number of people working, drinking coffee, 
reading at their computers.

Two of these people are a YOUNG MAN and a YOUNG WOMAN. The
Young Man has a slight working-class English accent.

             YOUNG MAN
  I want to tell you about what we've been studying.

             YOUNG WOMAN
  Oh? What have you been focusing on? 

             YOUNG MAN
  {expand_this}. For example"""


for x in range(generate):
  try:
    print(infer(top_p=top_p, 
                temp=temp, 
                gen_len=gen_len, 
                context=context.format(expand_this = expand_this))[0])
  except:
    pass

In [ ]:
#@title Make a title sound more academic 
#@markdown  
improve_this = "we're going ti make an instrument using the potentiometer " #@param {type:"string"}
#@markdown - - -
top_p = 0.9 #@ param {type:"slider", min:0, max:1, step:0.1}
temp = 0.9 #@ param {type:"slider", min:0, max:1, step:0.1}
gen_len = 64 #@ param {type: "slider", min:64, max: 2048, step: 64}
generate = 12 # @ param {type: "slider", min:1, max: 12, step: 1}

context = """
Instead of saying, "{improve_this}" a better shibboleth would be, \""""

#Indie Hackers want to click on titles such as 
#use something with more punch

for x in range(generate):
  try:
    print(infer(top_p=top_p, 
                temp=temp, 
                gen_len=gen_len, 
                context=context.format(improve_this = improve_this))[0].split('"')[3])
  except:
    pass

In [ ]:
str_list = [None, '', 0, "Hi", ' ', "Hello"]
[x for x in str_list if x != '']
[None, 0, "Hi", "Hello"]

In [ ]:
#@title Give headlines some more Clickbait 
improve_this = "Make an instrument using the a light sensor " #@param {type:"string"}
#@markdown - - - - - - - - - - - - - - - - - - - - - - - - 
top_p = 0.9 #@ param {type:"slider", min:0, max:1, step:0.1}
temp = 1 #@ param {type:"slider", min:0, max:1, step:0.1}
gen_len = 64 # @ param {type: "slider", min:64, max: 2048, step: 64}
generate = 5 #@ param {type: "slider", min:1, max: 12, step: 1}

context = f"""
You need to make your title more clickbate. So instead of,
"{improve_this}" use something with more punch such as,\""""

import codecs

def yield_quoted_text(top_p=top_p, 
                       temp=temp, 
                       gen_len=gen_len, 
                       context=context):
  while True:
    inference = infer(top_p=top_p, 
                       temp=temp, 
                       gen_len=gen_len, 
                       context=context)[0].split('"')[3].strip()
    inference = repr(inference).split('\\x1b[0m')
    inference = inference[1]
    if inference !="'":
      yield inference

#for x in range(generate):
#    while not got_a_headline:
for x in range(generate):
  headline = next(yield_quoted_text())
  print(headline)


In [ ]:
# @ title Generating inference prompts using tracery
import tracery
from tracery.modifiers import base_english

rules = {
    
    'origin': 'It was a #type_of_day# #time# in #location# and',
    'time': ['night', 'day', 'week', 'year'],
    'type_of_day': ['amazing', 'spectaculare' , 'horid', 'spooky' ],
    'location': ['Montreal', 'the woods', 'my head']
}

grammar = tracery.Grammar(rules)
grammar.add_modifiers(base_english)

for x in range(generate):
  try:
    print(infer(top_p=top_p, 
                temp=temp, 
                gen_len=gen_len, 
                context=grammar.flatten("#origin#"))[0] + "\n")
  except:
    pass

In [ ]:
# @ title Generating inference prompts using tracery
import tracery
from tracery.modifiers import base_english

rules = {
    
    'origin': 'Our team builds #imagine# new online #location# beyond',
    'imagine': ['amazing', 'spectaculare' , 'bold', ],
    'location': ['worlds', 'spaces', 'colabortion']
}

grammar = tracery.Grammar(rules)
grammar.add_modifiers(base_english)

for x in range(generate):
  try:
    print(infer(top_p=top_p, 
                temp=temp, 
                gen_len=gen_len, 
                context=grammar.flatten("#origin#"))[0] + "\n")
  except:
    pass

In [ ]:
#@title Generating grading comment inference prompts using tracery
import tracery
from tracery.modifiers import base_english

rules = {
    
    'origin': '#this_semester# Jane #does# #good_bad# in class. We hope that she',
    'this_semester': ['Our focus this semester was learning to program microcomputers, using sensors, and building electronic circuits.','This semester we focused on learning to build electric circuits, reading information from sensors, and programming microcontrollers.'],
    'does': ['works', 'does' , 'particpates', 'performs' ],
    'good_bad': ['well', 'competently', 'baddly']
}

grammar = tracery.Grammar(rules)
grammar.add_modifiers(base_english)

for x in range(generate):
  try:
    print(infer(top_p=top_p, 
                temp=temp, 
                gen_len=gen_len, 
                context=grammar.flatten("#origin#"))[0] + "\n")
  except:
    pass

In [ ]:
#@title Article Generator 
title = "Being your own Boss" #@param {type:"string"}
site = "" #@param {type:"string"}
date = "" #@param {type:"string"}
author = "Jim Rohn" #@param {type:"string"}
tags = "goals, drive, gtd" #@param {type:"string"}
lead = "## Table of Content" #@param {type:"string"}
#@markdown - - -
generate = 1

context = """
# {title}
### Published:{date}
### By:{author}
##### Tags: {tags}
### Site: {site}

{lead}
"""

for x in range(generate):
  print(infer(top_p=top_p, 
              temp=temp, 
              gen_len=gen_len, 
              context=context.format(title =title, 
                                     site = site, 
                                     date = date, 
                                     author = author, 
                                     tags = tags, 
                                     lead = lead))[0])

In [ ]:
#@title Article generation { form-width: "300px" }
title = "COVID-19: What do vaccine passports in Quebec & Ontario mean for Canada's largest provinces? " #@param {type:"string"}
site = "https://www.canada.ca/en/public-health/services/diseases/" #@param {type:"string"}
date = "Sep 7, 2021" #@param {type:"string"}
author = "" #@param {type:"string"}
tags = "COVID-19, Quebec, Ontario, Canada" #@param {type:"string"}
lead = "The passports are certificates that confirm vaccinations and allow people to enjoy eating" #@param {type:"string"}


#print("Enter/Paste your content. Ctrl-D or Ctrl-Z ( windows ) to save it.")
#context = []
#while True:
#    try:
#        line = input()
#    except EOFError:
#        break
#    context.append(line)

context = """
# {title}
### Published:{date}
### By:{author}
##### Tags: {tags}
### Site: {site}

{lead}"""

for x in range(generate):
  print(infer(top_p=top_p, 
              temp=temp, 
              gen_len=gen_len, 
              context=context.format(title =title, 
                                     site = site, 
                                     date = date, 
                                     author = author, 
                                     tags = tags, 
                                     lead = lead))[0])

## Code Generation

### Set Sampling Parameters


In [ ]:
#@markdown ### Leave this alone unless you know what you are doing ... or are just having fun experimenting.
#@markdown 
#@markdown - - - 
#@markdown Top_p controls diversity/quality. The higher our top_p the more rich and nuanced oir inferences can be.
top_p = 0.9 #@param {type:"slider", min:0, max:1, step:0.1}
#@markdown - - - 
#@markdown Temp controls randomnes. Low temps make for mechanical inferences.
temp = 0.9 #@param {type:"slider", min:0, max:1, step:0.1}
#@markdown - - - 
#@markdown Gen_len adjects the length of text infered.
#@markdown Specifcily it adustes the nember of tokens (~4 per engish word).
#@markdown
#@markdown *Note: Changing gen_len will cause long wait times when first setting lengs that have not been set before.*
#@markdown  
gen_len = 512 #@param {type: "slider", min:64, max: 1024, step: 64}
#@markdown - - - 
#@markdown Generate controls the number of inferences that will be generated.
generate = 1 #@param {type: "slider", min:1, max: 12, step: 1}


### Python Code

In [ ]:
#@title Python Code - Simple 
#@markdown Generate code by just writng a comment and a name for a fuction 
python_comment = "check if a number is prime or not" #@param {type:"string"}
#variable_n_val = "youtube_video_url" #@param {type:"string"}
function_name = "p" #@param {type:"string"}


context = """# {python_comment}

def is_""".format(python_comment = python_comment, 
           function_name = function_name)

for x in range(generate):
  print(infer(top_p=top_p, temp=temp, gen_len=gen_len, context=context)[0])

In [ ]:
#@title Python Code - Complex
python_comment = "detect security flaws " #@param {type:"string"}
variable_name ="person_real_name" #@param {type:"string"}
variable_val = "Isabelle Plante" #@param {type:"string"}
function_name = "online_account_scan" #@param {type:"string"}
#@markdown - - -


context = '''#!/usr/bin/env python3

# {python_comment}

{variable_name} = "{variable_val}"

def {function_name}(n):
  """This'''.format(python_comment = python_comment,
           variable_name = variable_name,
           variable_val = variable_val,
           function_name = function_name)

for x in range(generate):
  print(infer(top_p=top_p, temp=temp, gen_len=gen_len, context=context)[0])

In [ ]:
#@title Vue Code - Simple { form-width: "500px" }
comment = "guest speakers" #@param {type:"string"}
#variable_n_val = "youtube_video_url" #@param {type:"string"}
name = "tab_1" #@param {type:"string"}
#@markdown - - - 
#@markdown Leave this alone unless you know ...
top_p = 0.8 #@param {type:"slider", min:0, max:1, step:0.1}
temp = 0.9 #@param {type:"slider", min:0, max:1, step:0.1}
gen_len = 512 #@param {type:"slider", min:64, max:1024, step:64}
generate = 1 #@param {type: "slider", min:1, max: 12, step: 1}

context = """
<!-- Vue template for {comment} page -->
<section>
  <div id="app" class="container">
    <div class="content">
      <div id="{name} " class=""".format(comment = comment, 
           name = name)
      
print(infer(top_p=top_p, temp=temp, gen_len=gen_len, context=context)[0])

### JSON Gen

In [ ]:
#@title Json Code - Simple { form-width: "500px" }
title = "upcoming conferences" #@param {type:"string"}
#variable_n_val = "youtube_video_url" #@param {type:"string"}
name_1 = "subtitle" #@param {type:"string"}
#@markdown - - - 
#@markdown Leave this alone, unless... you know ...
top_p = 0.9 # @ param {type:"slider", min:0, max:1, step:0.1}
temp = 0.98 # @ param {type:"slider", min:0, max:1, step:0.1}
gen_len = 96 # @ param {type:"slider", min:64, max:1024, step:64}
generate = 1 # @ param {type: "slider", min:1, max: 12, step: 1}

context = f"""{{"{title}": [
  {{"{name_1}":"""

inference = infer(top_p=top_p, temp=temp, gen_len=gen_len, context=context)[0]

first_key  = ('{"' + name_1)

inference = (first_key.join((inference.split(first_key)[0:-1])))
print(inference)

In [ ]:
#@title Extend last infrence
#@markdown If you are happy with the last infrence run this to extend it.
extended_inference = infer(
    top_p=0.9, 
    temp=1.2, 
    gen_len=512, 
    context=inference)[0]
print(extended_inference)

# Examples of GPT genereated code.

The following are examples of code generated in prior runs.

Please paste interesting examples below

In [ ]:
# check if a number is prime or not

def is_prime(n):
  """ 
  > returns true if n is a prime,
  > false otherwise.
  """
  for i in range(2, n):
    if n % i == 0:
      return False
  return True

print(is_prime(7))

In [ ]:
# Program to check if a number is prime or not

num = 9

# Program to check if a number is prime or not

def is_prime(n):
    while n > 1:
        while n % 2 == 0:
            return False
        n = n / 2
    return True

print("This is an example of a function that works but is quite wrong!")

for i in range(2, 20):
  print(f"Is {i} prime? {is_prime(i)}")



In [ ]:
# check if a number is prime or not
def is_prime(n):
    for i in range(2, n):
        if n % i == 0:
            return False
    return True

is_prime(9)

In [ ]:
!pip install grip

In [ ]:
!pip install grip
from google.colab.output import eval_js
print(eval_js("google.colab.kernel.proxyPort(8000)"))
!grip 8000

In [ ]:
!pip install flask